In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np

In [ ]:
from model2vec import StaticModel

# Load a model from the HuggingFace hub (in this case the potion-base-8M model)
model = StaticModel.from_pretrained("minishlab/potion-base-8M")
qs = ["It's dangerous to go alone!", "I like green apples"]
negs = ["Danger is imminent when solo. ", "My favourite fruit is sweet and round"]
poss = ["I like cats", "Walking is slow"]
# Make embeddings
question_embeds = model.encode(qs)
pos_embeds = model.encode(poss)
neg_embeds = model.encode(negs)

In [ ]:
question_embeds.shape

In [ ]:
pos_dists = np.linalg.norm(pos_embeds - question_embeds, axis=1)
neg_dists = np.linalg.norm(neg_embeds - question_embeds, axis=1)

In [ ]:
predictions = (pos_dists < neg_dists).astype(int)
predictions

## BM25

In [ ]:
model.tokenizer.encode("I love cake ").tokens

Looks good.

In [ ]:
from rank_bm25 import BM25Okapi


corpi = [
    model.tokenizer.encode(pos_n + neg_n).tokens for pos_n, neg_n in zip(poss, negs)
]
corpi

In [ ]:
model.tokenizer.encode_batch(qs)

In [ ]:
tokenized_queries = [ts.tokens for ts in model.tokenizer.encode_batch(qs)]


bm25 = BM25Okapi(corpi)
scores = [bm25.get_scores(q) for q in tokenized_queries]

In [ ]:
bm25 = BM25Okapi(corpi)

In [ ]:
bm25.doc_freqs

In [ ]:
bm25.get_scores(model.tokenizer.encode("I like fruit").tokens)

In [ ]:
from datasets import load_dataset
import numpy as np

ds = load_dataset("DDSC/nordic-embedding-training-data")

In [ ]:
dsdk = ds.filter(lambda sample: True if sample["language"] == "danish" else False)
dsdk = dsdk["train"]
dsdk_with_idx = dsdk.add_column("idx", np.arange(len(dsdk)))

In [ ]:
# Sentence transformer

In [ ]:
from dkmodel2vec.evaluator import get_detailed_instruct
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer("intfloat/multilingual-e5-large-instruct")

In [ ]:
dsdk

In [ ]:
dataset = dsdk.map(get_detailed_instruct)

In [ ]:
model.encode("flkajsd")